In [4]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [6]:
# List of file names
file_names = ['Output\output_1_1.csv', 'Output\output_1_3.csv', 'Output\output_1_5.csv', 'Output\output_1_8.csv', 'Output\output_1_10.csv', 'Output\output_2_2.csv', 'Output\output_2_10.csv']

# Load and merge the CSV files
df_list = [pd.read_csv(file_name) for file_name in file_names]
merged_df = pd.concat(df_list, ignore_index=True)

In [4]:
# Specify the column in which to look for NaN values
column_with_nan = 'h3_geography_freq'  # Replace 'A' with the column name you're interested in

# Remove rows where the specified column has NaN
df_cleaned = merged_df.dropna(subset=[column_with_nan])

In [5]:
df_copy = df_cleaned.copy()

In [59]:
dataset = df_copy.iloc[:, :35]
dataset = dataset.dropna()

In [61]:
# Concatenating the values of columns 3 and 4
dataset['group_id'] = dataset.iloc[:, [2, 3]].astype(str).agg('-'.join, axis=1)

# Creating unique IDs
dataset['group_id'] = pd.factorize(dataset['group_id'])[0]

In [ ]:
# Counting the number of unique values in the 'group_id' column
unique_group_ids = dataset['group_id'].nunique()

print("Number of unique group IDs:", unique_group_ids)

In [63]:
rel_dataset = dataset.copy()

In [64]:
import numpy as np
import pandas as pd

# Assuming rel_dataset is your DataFrame

# Find the row indices with the lowest classifier value for each group
best_classified_indices = rel_dataset.groupby('group_id')[rel_dataset.columns[1]].idxmin()

# For each group, divide values in columns 5-35 by the best-classified row's values, if not zero
for group_id, best_idx in best_classified_indices.iteritems():
    # Select columns 5-35 for the current group
    group_rows = rel_dataset.loc[rel_dataset['group_id'] == group_id, rel_dataset.columns[4:35]]

    # Get the best-classified row's values
    best_values = rel_dataset.loc[best_idx, rel_dataset.columns[4:35]]

    # Perform division, but avoid division by zero
    for col in rel_dataset.columns[4:35]:
        best_value = best_values[col]
        if best_value != 0:
            rel_dataset.loc[rel_dataset['group_id'] == group_id, col] = group_rows[col] / best_value



In [10]:
import numpy as np

# Assuming df is your DataFrame

# Find the row indices with the lowest classifier value for each group
best_classified_indices = rel_dataset.groupby('group_id')[rel_dataset.columns[1]].idxmin()

# For each group, divide values in columns 5-35 by the best-classified row's values
for group_id, best_idx in best_classified_indices.iteritems():
    # Select columns 5-35 for the current group
    group_rows = rel_dataset[rel_dataset['group_id'] == group_id][rel_dataset.columns[4:35]]

    # Get the best-classified row's values, replace 0s with a small number to avoid division by zero
    best_values = rel_dataset.loc[best_idx, rel_dataset.columns[4:35]].replace(0, 1e-10)

    # Divide each row by the best-classified row's values and update the DataFrame
    rel_dataset.loc[rel_dataset['group_id'] == group_id, rel_dataset.columns[4:35]] = group_rows.div(best_values)


In [17]:
rel_dataset.head(30)

,url,rank_group,keyword,geography,h1_tags_amount,h1_avg_char_count,h1_keyword_freq,h1_geography_freq,h2_tags_amount,h2_keyword_freq,...,alt_keyword_freq,alt_geography_freq,a_keyword_freq,a_geography_freq,url_keyword_freq,url_geography_freq,p_total_char_count,p_total_keyword_freq,p_total_geography_freq,group_id
0,https://www.booking.com/city/se/stockholm.sv.html,1,Hotell,Stockholm,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,https://sv.hotels.com/de1270279/hotell-i-stock...,2,Hotell,Stockholm,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,https://www.scandichotels.se/hotell/sverige/st...,3,Hotell,Stockholm,0.0,0.0,0.0,0.0,0.272727,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.088826,0.0,0.0,0
3,https://www.elite.se/hotell/stockholm/,4,Hotell,Stockholm,1.0,0.84375,0.0,0.0,1.818182,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.803162,0.0,0.0,0
4,https://www.strawberry.se/hotell/sverige/stock...,5,Hotell,Stockholm,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5,https://www.agoda.com/sv-se/city/stockholm-se....,6,Hotell,Stockholm,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,https://www.booking.com/region/se/stockholm.sv...,7,Hotell,Stockholm,1.0,1.4375,0.0,0.0,1.272727,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.332919,0.0,0.0,0
7,https://www.tripadvisor.se/Hotels-g189852-Stoc...,8,Hotell,Stockholm,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,https://www.bestwestern.se/hotell-stockholm,9,Hotell,Stockholm,1.0,0.5625,0.0,0.0,1.181818,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.702789,0.0,0.0,0
9,https://www.diplomathotel.com/,10,Hotell,Stockholm,1.0,0.75,0.0,0.0,0.272727,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.185735,0.0,0.0,0


In [18]:
import pandas as pd

# Assuming df is your DataFrame

# Count the number of columns where all values are zero
num_zero_columns = (dataset == 0).all().sum()

print(f"Number of columns with only zero values: {num_zero_columns}")


Number of columns with only zero values: 1


In [12]:
# Calculate the correlation matrix
correlation_matrix = rel_dataset.corr()

print(correlation_matrix)

                        rank_group  h1_tags_amount  h1_avg_char_count  \
rank_group                1.000000        0.005715           0.016303   
h1_tags_amount            0.005715        1.000000           0.197273   
h1_avg_char_count         0.016303        0.197273           1.000000   
h1_keyword_freq           0.000154       -0.001290          -0.002270   
h1_geography_freq         0.001552       -0.002766          -0.004235   
h2_tags_amount           -0.002708        0.090516           0.138849   
h2_keyword_freq          -0.003282       -0.000408          -0.000448   
h2_geography_freq        -0.000905       -0.001887          -0.003283   
h3_tags_amount            0.004798        0.051019           0.089736   
h3_keyword_freq          -0.001259       -0.000113           0.000029   
h3_geography_freq        -0.000529       -0.000437          -0.000091   
h4_tags_amount            0.000893        0.010621           0.019305   
p_tags_amount             0.001956        0.049943 

In [29]:
import pandas as pd

# Assuming df is your DataFrame

# Count the number of columns where all values are zero
num_zero_columns = (rel_dataset == 0).all().sum()

print(f"Number of columns with only zero values: {num_zero_columns}")


Number of columns with only zero values: 1


In [31]:
# Calculate the correlation matrix
correlation_matrix = rel_dataset.corr()

print(correlation_matrix)

                        rank_group  h1_tags_amount  h1_avg_char_count  \
rank_group                1.000000        0.005715           0.016303   
h1_tags_amount            0.005715        1.000000           0.197273   
h1_avg_char_count         0.016303        0.197273           1.000000   
h1_keyword_freq           0.000154       -0.001290          -0.002270   
h1_geography_freq         0.001552       -0.002766          -0.004235   
h2_tags_amount           -0.002708        0.090516           0.138849   
h2_keyword_freq          -0.003282       -0.000408          -0.000448   
h2_geography_freq        -0.000905       -0.001887          -0.003283   
h3_tags_amount            0.004798        0.051019           0.089736   
h3_keyword_freq          -0.001259       -0.000113           0.000029   
h3_geography_freq        -0.000529       -0.000437          -0.000091   
h4_tags_amount            0.000893        0.010621           0.019305   
p_tags_amount             0.001956        0.049943 

In [65]:
# Filter columns that contain '_freq'
freq_columns = rel_dataset.filter(like='_freq').columns

# Drop these columns
rel_dataset_cleaned = rel_dataset.drop(freq_columns, axis=1)

In [67]:
# Extracting the classifier (target)
y = rel_dataset_cleaned.iloc[:, 1]  # Classifier is in the second column (index 1)

# Extracting the feature values
X = rel_dataset_cleaned.iloc[:, 4:19]  # Features are from the 5th to the 35th column (index 4 to 34)
X = X.astype(float)

In [22]:
X.head()

,h1_tags_amount,h1_avg_char_count,h2_tags_amount,h3_tags_amount,h4_tags_amount,p_tags_amount,a_internal_tags,a_external_tags,img_tags_amount,meta_tags_amount,title_word_count,description_word_count,title_char_count,description_char_count,p_total_char_count
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.272727,0.0625,0.0,0.085714,2.95082,0.165746,0.055556,0.333333,1.222222,1.037037,1.361702,1.0,0.088826
3,1.0,0.84375,1.818182,0.425,20.5,0.828571,5.377049,0.055249,0.756944,0.27451,1.111111,0.962963,1.212766,0.890173,1.803162
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005525,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
from sklearn.tree import DecisionTreeClassifier

# Create a decision tree classifier with entropy criterion
clf = DecisionTreeClassifier(criterion='entropy')
clf.fit(X, y)

# Get feature importances
feature_importances = clf.feature_importances_

# Create a DataFrame for easier interpretation
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort by importance
feature_importance_df.sort_values(by='Importance', ascending=False, inplace=True)

print(feature_importance_df)

                   Feature  Importance
12        title_char_count    0.099048
9         meta_tags_amount    0.092130
7          a_external_tags    0.089475
8          img_tags_amount    0.083165
14      p_total_char_count    0.078192
1        h1_avg_char_count    0.075839
5            p_tags_amount    0.073221
6          a_internal_tags    0.070675
10        title_word_count    0.063992
13  description_char_count    0.058200
2           h2_tags_amount    0.057521
3           h3_tags_amount    0.053484
11  description_word_count    0.049701
4           h4_tags_amount    0.031957
0           h1_tags_amount    0.023401


                   Feature  Importance
12        title_char_count    0.099258
9         meta_tags_amount    0.091928
7          a_external_tags    0.089576
8          img_tags_amount    0.083315
14      p_total_char_count    0.077822
1        h1_avg_char_count    0.075958
5            p_tags_amount    0.073404
6          a_internal_tags    0.070461
10        title_word_count    0.063588
13  description_char_count    0.058269
2           h2_tags_amount    0.057644
3           h3_tags_amount    0.053365
11  description_word_count    0.049992
4           h4_tags_amount    0.031996
0           h1_tags_amount    0.023426

                   Feature  Importance
14      p_total_char_count    0.104825
12        title_char_count    0.098812
7          a_external_tags    0.095775
1        h1_avg_char_count    0.084851
8          img_tags_amount    0.076540
5            p_tags_amount    0.075344
13  description_char_count    0.074810
6          a_internal_tags    0.070249
9         meta_tags_amount    0.069296
11  description_word_count    0.056112
3           h3_tags_amount    0.051822
10        title_word_count    0.049183
2           h2_tags_amount    0.048686
4           h4_tags_amount    0.030501
0           h1_tags_amount    0.013192


In [40]:
X.head()

,h1_tags_amount,h1_avg_char_count,h2_tags_amount,h3_tags_amount,h4_tags_amount,p_tags_amount,a_internal_tags,a_external_tags,img_tags_amount,meta_tags_amount,title_word_count,description_word_count,title_char_count,description_char_count,p_total_char_count
0,1.0,1.00000,1.000000,1.0000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.0,0.00000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.00000,0.272727,0.0625,0.0,0.085714,2.950820,0.165746,0.055556,0.333333,1.222222,1.037037,1.361702,1.000000,0.088826
3,1.0,0.84375,1.818182,0.4250,20.5,0.828571,5.377049,0.055249,0.756944,0.274510,1.111111,0.962963,1.212766,0.890173,1.803162
4,0.0,0.00000,0.000000,0.0000,0.0,0.000000,0.000000,0.005525,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [87]:
# Define a function to map values
def map_to_smaller_range(value):
    return (value - 1) // 10 + 1

# Apply the function to the 'classifiers' column
y = y.apply(map_to_smaller_range)

In [68]:
# Define a function to map values to the specified ranges
def map_to_smaller_range(value):
    if value <= 5:
        return 1  # Top 5
    elif value <= 10:
        return 2  # 6-10
    elif value <= 20:
        return 3  # 11-20
    elif value <= 30:
        return 4  # 21-40
    elif value <= 30:
        return 5  # 41-70
    else:
        return 6  # 71-100

# Apply the function to the 'classifiers' column
y = y.apply(map_to_smaller_range)

In [44]:
y.head(20)

0     1
1     1
2     1
3     1
4     1
5     2
6     2
7     2
8     2
9     2
10    3
11    3
12    3
13    3
14    3
15    3
16    3
17    3
18    3
19    3
Name: rank_group, dtype: int64

In [ ]:
# Assuming y is your modified target series with the new categorical values

# Count the occurrences of each category and normalize to get proportions
category_counts = y.value_counts(normalize=True)

# Convert proportions to percentages
category_percentages = category_counts * 100

print(category_percentages)


In [70]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Decision Tree Classifier
clf = DecisionTreeClassifier()

# Train the model on the training set
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [72]:
import catboost as cb
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
# df = pd.read_csv('your_dataset.csv')  # Replace with your dataset path
# X = df.drop('target', axis=1)  # Replace 'target' with your target variable
# y = df['target']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a CatBoost classifier
model = cb.CatBoostClassifier()

# Train the model
model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

# The model is now trained and can be used to make predictions or further analysis


In [73]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Test Accuracy: {accuracy:.2f}")


Test Accuracy: 0.68


In [69]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train -= 1
y_test -= 1

# Define the XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', 
                            num_class=3,
                            colsample_bytree=0.3, 
                            learning_rate=0.1,
                            max_depth=5, 
                            alpha=10, 
                            n_estimators=100)

# Train the model
xgb_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.674952213471978


In [74]:
# Import necessary libraries
import numpy as np
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

# Subtract 1 from class labels to make them 0-indexed
y_train -= 1
y_test -= 1

from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.7, 0.9],
    'colsample_bytree': [0.5, 0.7, 0.9],
    'n_estimators': [100, 200, 300]
}

# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_clf, param_grid=param_grid, cv=3, scoring='accuracy')

# Perform grid search
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Train the model with the best parameters
best_xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3, **best_params)
best_xgb_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_xgb_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)




Best Parameters: {'colsample_bytree': 0.9, 'learning_rate': 0.3, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.9}
Accuracy: 0.6804419298111476
